In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
home_dir = "../../"

import os
import pandas as pd

### Population NP_ids mapping to NM_ids

In [3]:
gene2refseq_mapping_filepath = home_dir+"data/refseq/MANE.GRCh38.v1.0.summary.txt.gz" # "symbol" is the Gene-symbol col
# gene2refseq_mapping_filepath = home_dir+"data/gene/gene2refseq_filtered.gz" # "Symbol" is the Gene-symbol col
# gene2refseq_mapping_filepath = home_dir+"data/refseq/HumanLRGRefSeqGene.txt.gz"  # "Symbol" is the Gene-symbol col

gene_symbol_col_name = "symbol" if gene2refseq_mapping_filepath.__contains__("MANE") else "Symbol"
gene2refseq_mapping_df = pd.read_csv(gene2refseq_mapping_filepath, compression='gzip', delim_whitespace=False, sep="\t")#, header=None, names=col_names)

print(gene2refseq_mapping_df.shape)
print(len(gene2refseq_mapping_df[gene_symbol_col_name].unique())) # some gene can be mapped to multiple proteins
gene2refseq_mapping_df.head()

(19120, 14)
19062


,#NCBI_GeneID,Ensembl_Gene,HGNC_ID,symbol,name,RefSeq_nuc,RefSeq_prot,Ensembl_nuc,Ensembl_prot,MANE_status,GRCh38_chr,chr_start,chr_end,chr_strand
0,GeneID:1,ENSG00000121410.12,HGNC:5,A1BG,alpha-1-B glycoprotein,NM_130786.4,NP_570602.2,ENST00000263100.8,ENSP00000263100.2,MANE Select,19,58345183,58353492,-
1,GeneID:2,ENSG00000175899.15,HGNC:7,A2M,alpha-2-macroglobulin,NM_000014.6,NP_000005.3,ENST00000318602.12,ENSP00000323929.8,MANE Select,12,9067708,9115919,-
2,GeneID:9,ENSG00000171428.15,HGNC:7645,NAT1,N-acetyltransferase 1,NM_000662.8,NP_000653.3,ENST00000307719.9,ENSP00000307218.4,MANE Select,8,18210109,18223689,+
3,GeneID:10,ENSG00000156006.5,HGNC:7646,NAT2,N-acetyltransferase 2,NM_000015.3,NP_000006.2,ENST00000286479.4,ENSP00000286479.3,MANE Select,8,18391282,18401218,+
4,GeneID:12,ENSG00000196136.18,HGNC:16,SERPINA3,serpin family A member 3,NM_001085.5,NP_001076.2,ENST00000393078.5,ENSP00000376793.3,MANE Select,14,94612391,94624053,+


In [6]:
from data_loader import get_population_freq_SNVs, get_protein_sequences

popu_protid_seq_tuple_list = get_protein_sequences(home_dir, max_seq_len=1022, return_type="protid_seq_tuple_list", data_type="popu_freq")
popu_protid_seq_df = pd.DataFrame(popu_protid_seq_tuple_list, columns =['prot_acc_version', 'seq'])

variants_df = get_population_freq_SNVs(home_dir=home_dir)
popu_merged_df = pd.merge(variants_df, gene2refseq_mapping_df, how="inner", left_on="prot_acc_version", right_on="RefSeq_prot")
popu_merged_df = popu_merged_df.rename(columns={"RefSeq_nuc": "nuc_acc_version"})
popu_prot_nuc_pair_df = popu_merged_df[["prot_acc_version", "nuc_acc_version"]].drop_duplicates(keep="first")

popu_prot_nuc_seq_df = pd.merge(popu_prot_nuc_pair_df, popu_protid_seq_df, how="inner", on="prot_acc_version")
popu_prot_nuc_seq_df


Log: Loading combined fasta iterator ...
#-protein sequences (seq-len<=1022): 15962

Log: Loading data ...
(95223, 13)
Index(['snp_id', 'chrom_acc_version', 'chrom_pos', 'ref_allele', 'alt_allele',
       'prot_acc_version', 'prot_pos', 'wt', 'mut', 'wt_population',
       'mut_poulation', 'wt_freq', 'mt_freq'],
      dtype='object')
After combining common (18279), rare (29383) and sampled-singletons (48434), data: (95223, 13)
NP_001277137.1    95
NP_001291317.1    78
NP_002115.2       74
NP_112241.2       68
NP_001075106.2    65
                  ..
NP_037408.2        1
NP_066951.1        1
NP_001013716.2     1
NP_001316544.1     1
NP_065171.2        1
Name: prot_acc_version, Length: 14997, dtype: int64


,prot_acc_version,nuc_acc_version,seq
0,NP_064519.2,NM_020134.4,MLANSASVRILIKGGKVVNDDCTHEADVYIENGIIQQVGRELMIPG...
1,NP_004276.2,NM_004285.4,MWNMLIVAMCLALLGCLQAQELQGHVSIILLGATGDLAKKYLWQGL...
2,NP_758872.1,NM_172362.3,MTMAGGRRGLVAPQNTFLENIVRRSNDTNFVLGNAQIVDWPIVYSN...
3,NP_065806.1,NM_020755.4,MGSVLGLCSMASWIPCLCGSAPCLLCRCCPSGNNSTVTRLIYALFL...
4,NP_001124389.2,NM_001130917.3,MTPILTVLICLGLSLGPRTHVQAGHLPKPTLWAEPGSVIIQGSPVT...
...,...,...,...
14992,NP_001012995.1,NM_001012977.3,MASLYVGDLHPEVTEAMLYEKFSPAGPILSIRICRDKITRRSLGYA...
14993,NP_061332.2,NM_018844.4,MTLQWAAVATFLYAEIGLILIFCLPFIPPQRWQKIFSFNVWGKIAT...
14994,NP_653194.1,NM_144593.3,MPLVRYRKVVILGYRCVGKTSLAHQFVEGEFSEGYDPTVENTYSKI...
14995,NP_003505.1,NM_003514.2,MSGRGKQGGKARAKAKTRSSRAGLQFPVGRVHRLLRKGNYAERVGA...


### Pathogenicity NP_ids mapping to NM_ids

In [7]:
gene2refseq_filtered_filepath = home_dir+"data/gene/gene2refseq_filtered.gz"
gene2refseq_filtered_df = pd.read_csv(gene2refseq_filtered_filepath, compression='gzip', comment='#', delim_whitespace=False, sep="\t")#, header=None, names=col_names)
print(gene2refseq_filtered_df.shape)
gene2refseq_filtered_df

(63738, 16)


,tax_id,GeneID,status,RNA_nucleotide_accession.version,RNA_nucleotide_gi,protein_accession.version,protein_gi,genomic_nucleotide_accession.version,genomic_nucleotide_gi,start_position_on_the_genomic_accession,end_position_on_the_genomic_accession,orientation,assembly,mature_peptide_accession.version,mature_peptide_gi,Symbol
0,9606,1,REVIEWED,NM_130786.4,1653960645,NP_570602.2,21071030,NC_000019.10,568815579,58345182,58353491,-,Reference GRCh38.p14 Primary Assembly,-,-,A1BG
1,9606,2,REVIEWED,NM_000014.6,1779421316,NP_000005.3,1779421317,NC_000012.12,568815586,9067707,9116228,-,Reference GRCh38.p14 Primary Assembly,-,-,A2M
2,9606,2,REVIEWED,NM_001347423.2,1677538580,NP_001334352.2,1677538581,NC_000012.12,568815586,9067707,9116228,-,Reference GRCh38.p14 Primary Assembly,-,-,A2M
3,9606,2,REVIEWED,NM_001347424.2,1779421277,NP_001334353.2,1779421278,NC_000012.12,568815586,9067707,9116228,-,Reference GRCh38.p14 Primary Assembly,-,-,A2M
4,9606,2,REVIEWED,NM_001347425.2,1779421327,NP_001334354.2,1779421328,NC_000012.12,568815586,9067707,9116228,-,Reference GRCh38.p14 Primary Assembly,-,-,A2M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63733,9606,122526780,VALIDATED,NM_001396008.1,2095179403,NP_001382937.1,2095179404,NC_000017.11,568815581,6636779,6640315,-,Reference GRCh38.p14 Primary Assembly,-,-,LOC122526780
63734,9606,122539214,VALIDATED,NM_001396016.1,2095530055,NP_001382945.1,2095530056,NC_000019.10,568815579,52650446,52690495,-,Reference GRCh38.p14 Primary Assembly,-,-,LOC122539214
63735,9606,123956252,VALIDATED,NM_001400774.1,2183686195,NP_001387703.1,2183686196,NC_000006.12,568815592,83853359,83967422,+,Reference GRCh38.p14 Primary Assembly,-,-,RIPPLY2-CYB5R4
63736,9606,124188214,VALIDATED,NM_001401340.1,2191823091,NP_001388269.1,2191823092,NC_000017.11,568815581,32895432,32906585,+,Reference GRCh38.p14 Primary Assembly,-,-,H2BN1


In [8]:
from data_loader import get_pathogenicity_analysis_SNVs, get_protein_sequences

def get_np_nm_pairs(pathogenicity_type):
    patho_protid_seq_tuple_list = get_protein_sequences(home_dir, max_seq_len=1022, return_type="protid_seq_tuple_list", data_type=pathogenicity_type)
    patho_protid_seq_df = pd.DataFrame(patho_protid_seq_tuple_list, columns =['prot_acc_version', 'seq'])

    list_of_variants_df = get_pathogenicity_analysis_SNVs(home_dir, pathogenicity_type)
    patho_df = list_of_variants_df[0]
    patho_df = patho_df[patho_df["class"] == pathogenicity_type]
    patho_merged_df = pd.merge(patho_df, gene2refseq_filtered_df, how="left", left_on="prot_acc_version", right_on="protein_accession.version")
    patho_merged_df = patho_merged_df.rename(columns={"RNA_nucleotide_accession.version": "nuc_acc_version"})
    print(patho_merged_df.shape)
    patho_prot_nuc_pair_df = patho_merged_df[["prot_acc_version", "nuc_acc_version"]].drop_duplicates(keep="first")
    patho_prot_nuc_seq_df = pd.merge(patho_prot_nuc_pair_df, patho_protid_seq_df, how="inner", on="prot_acc_version")
    return patho_prot_nuc_seq_df


patho_prot_nuc_seq_df = get_np_nm_pairs("pathogenic")
print(patho_prot_nuc_seq_df.head())
likely_patho_prot_nuc_seq_df = get_np_nm_pairs("likely_pathogenic")
print(likely_patho_prot_nuc_seq_df.head())


Log: Loading combined fasta iterator ...
#-protein sequences (seq-len<=1022): 908

Log: Loading combined fasta iterator ...
pathogenic raw data: (3579, 12)
Index(['clinvar_id', 'gene_symbol', 'gene_id', 'chrom_acc_version',
       'chrom_pos', 'ref_allele', 'alt_allele', 'prot_acc_version', 'prot_pos',
       'wt', 'mut', 'class'],
      dtype='object')

Log: excluding variants corresponding to proteins having seq-len>1022 ...

Log: Loading combined fasta iterator ...
#-protein sequences (seq-len<=1022): 908
(2293, 12)

Log: Loading population freq variants dataset ...
Index(['snp_id', 'chrom_acc_version', 'chrom_pos', 'ref_allele', 'alt_allele',
       'prot_acc_version', 'prot_pos', 'wt', 'mut', 'wt_population',
       'mut_poulation', 'wt_freq', 'mt_freq'],
      dtype='object')
0 (4586, 10)
1 (4586, 10)
2 (4586, 10)
3 (4586, 10)
4 (4586, 10)
5 (4586, 10)
6 (4586, 10)
7 (4586, 10)
8 (4586, 10)
9 (4586, 10)
(2296, 26)
  prot_acc_version nuc_acc_version  \
0   NP_001381642.1  NM_0013

### Concatenating all NP_ids mapping to NM_ids

In [14]:
prot_nuc_seq_df = pd.concat([popu_prot_nuc_seq_df, patho_prot_nuc_seq_df, likely_patho_prot_nuc_seq_df])
print(prot_nuc_seq_df.shape)
prot_nuc_seq_df = prot_nuc_seq_df.drop_duplicates(keep="first")
prot_nuc_seq_df.to_csv(home_dir+"models/aa_common/merged_np_nm_seq/merged_np_nm_seq.csv", sep="\t", header=True, index=False)
prot_nuc_seq_df


(16982, 3)


,prot_acc_version,nuc_acc_version,seq
0,NP_064519.2,NM_020134.4,MLANSASVRILIKGGKVVNDDCTHEADVYIENGIIQQVGRELMIPG...
1,NP_004276.2,NM_004285.4,MWNMLIVAMCLALLGCLQAQELQGHVSIILLGATGDLAKKYLWQGL...
2,NP_758872.1,NM_172362.3,MTMAGGRRGLVAPQNTFLENIVRRSNDTNFVLGNAQIVDWPIVYSN...
3,NP_065806.1,NM_020755.4,MGSVLGLCSMASWIPCLCGSAPCLLCRCCPSGNNSTVTRLIYALFL...
4,NP_001124389.2,NM_001130917.3,MTPILTVLICLGLSLGPRTHVQAGHLPKPTLWAEPGSVIIQGSPVT...
...,...,...,...
1023,NP_001340850.1,NM_001353921.2,MQWIRGGSGMLITGDSIVSAEAVWDHVTMANRELAFKAGDVIKVLD...
1041,NP_002755.1,NM_002764.4,MPNIKIFSGSSHQDLSQKIADRLGLELGKVVTKKFSNQETCVEIGE...
1058,NP_004105.1,NM_004114.5,MAAAIASSLIRQKRQAREREKSNACKCVSSPSKGKTSCDKNKLNVF...
1068,NP_003482.1,NM_003491.4,MNIRNARPEDLMNMQHCNLLCLPENYQMKYYFYHGLSWPQLSYIAE...
